In [1]:
import pandas as pd 
import numpy as np

In [2]:
admissions=pd.read_csv("admissions.csv")
admissions['actual_label']=admissions['admit']
admissions.head()

,admit,gpa,gre,actual_label
0,0,3.177277,594.102992,0
1,0,3.412655,631.528607,0
2,0,2.728097,553.714399,0
3,0,3.093559,551.089985,0
4,0,3.141923,537.184894,0


In [3]:
admissions=admissions.drop('admit',axis=1)

In [4]:
shuffled_index=np.random.permutation(admissions.index)

In [8]:
shuffled_admissions=admissions.loc[shuffled_index] 

In [9]:
shuffled_admissions.head()

,gpa,gre,actual_label
468,3.645978,749.877819,1
431,3.556059,589.044756,1
546,3.581406,559.764878,1
404,3.307681,715.000076,1
544,3.163068,592.303659,1


In [10]:
admissions=shuffled_admissions.reset_index()

In [11]:
admissions.head()

,index,gpa,gre,actual_label
0,468,3.645978,749.877819,1
1,431,3.556059,589.044756,1
2,546,3.581406,559.764878,1
3,404,3.307681,715.000076,1
4,544,3.163068,592.303659,1


In [14]:
admissions.loc[0:128,'fold']=1 

In [15]:
admissions.head()

,index,gpa,gre,actual_label,fold
0,468,3.645978,749.877819,1,1.0
1,431,3.556059,589.044756,1,1.0
2,546,3.581406,559.764878,1,1.0
3,404,3.307681,715.000076,1,1.0
4,544,3.163068,592.303659,1,1.0


In [16]:
admissions.loc[129:257,'fold']=2 

In [18]:
admissions.loc[258:386,'fold']=3 
admissions.loc[387:514,'fold']=4
admissions.loc[515:644,'fold']=5 

In [19]:
admissions['fold']=admissions['fold'].astype('int')

In [20]:
print(admissions.head())

   index       gpa         gre  actual_label  fold
0    468  3.645978  749.877819             1     1
1    431  3.556059  589.044756             1     1
2    546  3.581406  559.764878             1     1
3    404  3.307681  715.000076             1     1
4    544  3.163068  592.303659             1     1


In [21]:
print(admissions.tail())

     index       gpa         gre  actual_label  fold
639    608  3.774355  717.463598             1     5
640    234  2.885002  510.426077             0     5
641    266  2.782338  558.012494             0     5
642    179  3.044987  564.255010             0     5
643    262  3.232388  533.307638             0     5


In [37]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
train_iteration_one=admissions[admissions['fold']!=5]
test_iteration_one=admissions[admissions['fold']==5]

In [38]:
len(train_iteration_one),len(test_iteration_one)

(515, 129)

In [39]:
model.fit(train_iteration_one[['gpa']],train_iteration_one['actual_label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
labels=model.predict(test_iteration_one[["gpa"]])

In [41]:
test_iteration_one['predicted_label']=labels

/Users/nanji/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
matches=test_iteration_one['predicted_label']==test_iteration_one['actual_label']

In [43]:
matches[:5]

515    False
516    False
517     True
518     True
519    False
dtype: bool

In [44]:
correct_predictions=test_iteration_one[matches]

In [45]:
correct_predictions[:5]

,index,gpa,gre,actual_label,fold,predicted_label
517,139,3.123864,610.015117,0,5,0
518,317,3.249892,490.932053,0,5,0
520,97,3.629889,559.226511,0,5,0
521,10,3.264341,636.453166,0,5,0
522,236,3.309465,538.928244,0,5,0


In [46]:
iteration_one_accuracy=len(correct_predictions)/float(len(test_iteration_one))

In [47]:
print(iteration_one_accuracy)

0.6046511627906976


In [48]:
import numpy as np
fold_ids=[1,2,3,4,5]

In [69]:
def train_and_test(df,folds):
    fold_accuracies=[]
    for fold in folds:
        model=LogisticRegression()
        train=admissions[admissions['fold']!=fold]
        test=admissions[admissions['fold']==fold]
        model.fit(train[['gpa']],train['actual_label'])
        labels=model.predict(test[['gpa']]) 
        test['predicted_label']=labels 
        matches=test['predicted_label']==test['actual_label']
        correct_predictions=test[matches]
        print(len(test),len(correct_predictions))
        fold_accuracies.append(len(correct_predictions)/float(len(test)))
    return (fold_accuracies)

accuracies=train_and_test(admissions,fold_ids)


129 69
129 89
129 84
128 85
129 78


/Users/nanji/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [70]:
print(accuracies)

[0.5348837209302325, 0.689922480620155, 0.6511627906976745, 0.6640625, 0.6046511627906976]


In [71]:
average_accuracy=np.mean(accuracies)
print(average_accuracy)

0.628936531008


In [72]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

In [73]:
admissions=pd.read_csv("admissions.csv")
admissions['actual_label']=admissions['admit']
admissions=admissions.drop('admit',axis=1)

In [77]:
kf=KFold(len(admissions),5,shuffle=True,random_state=8)
lr=LogisticRegression()
#roc_auc
accuracies=cross_val_score(lr,admissions[['gpa']],admissions['actual_label'],scoring='roc_auc',cv=kf)
average_accuracy=sum(accuracies)/len(accuracies)



In [78]:
print(accuracies)

[ 0.70790123  0.69550265  0.65987934  0.73363017  0.57864583]


In [79]:
print(average_accuracy)

0.675111844524
